# Downloading Data & Save them on Google Drive
we are using Persian commonvoice mozilla dataset from  'https://commonvoice.mozilla.org/fa/datasets'

In [ ]:
dataset_link = 
! wget -O dataset_file.zip dataset_link
! unzip dataset_file

then we save test, train, dev files and transcripts and get rid of useless files

# Data cleaning


In [ ]:
import pandas as pd
train_ds = pd.read_csv("/content/drive/MyDrive/train.tsv", delimiter="\t")
validation_ds = pd.read_csv("/content/drive/MyDrive/dev.tsv", delimiter="\t")

we extract the vocab from dev and train transcripts, and we try to clean it

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

all_sentences = np.concatenate((train_sents, val_sents),axis=0)
all_sentences = ' '.join(all_sentences)

sentences_tokens = Tokenizer(char_level=True)
sentences_tokens.fit_on_texts([all_sentences])

sentences_tokens.word_index.keys()

clearing out vocab from bad characters

In [ ]:
char_type_swap ={
          ord("ى")   :ord("ی"),
          ord("ﯽ")   : ord("ی"),
          ord("ﺩ")    :ord("د"),
          ord("ﻮ")     :ord("و"),
          ord("ﻢ")     :ord("م"),
          ord("ﺍ")     :ord("ا"),
          ord("ﺱ")    :ord("س"),
          ord("ﻡ")    :ord("م"),
          ord('ﺘ')    : ord("ت"),
          ord('ە')    : ord("ه"),
          ord('ﮐ')    : ord("ک"),
          ord('ﺧ')   : ord("خ"),
          ord('ﭘ')    : ord("پ"),
          ord('ﺑ')    : ord("ب"),
          ord('ﻀ')   : ord("ض"),
          ord('ﻌ')    : ord("ع"),
          ord('ﺧ')    : ord("خ"),
          ord('ﺘ')    : ord("ت"),
          ord('ﻪ')    : ord("ه"),
          ord('ﻟ')    : ord("ل"),
          ord("ﺎ")    : ord("ا")
}
arabic_to_persian = {
          ord('ة')  : ord('ه'),
			    ord('ك')  : ord('ک'),
          ord('ي')  : ord('ی'),
          ord('ئ')  : ord('ی'),
			    ord('أ')   : ord('ا'),
          ord('ۀ')   : ord('ه'),
          ord('ؤ')   : ord('و'),
          ord('ے')  : ord('ی'),
}
special_chars_clear = {
          ord('"')     : None,
          ord('\'')    : None,
          ord('ـ')     : None,
          ord('َ')      : None,
          ord('ُ')      : None,
          ord('ِ')      : None,
          ord('ّ')      : None,
          ord('š')    : None,
          ord('ā')    : None,
          ord(':')    : None,
          ord('٬')    : None,
          ord('«')    : None,
          ord('»')    : None,
          ord(')')    : None,
          ord('(')    :None,
          ord(';')    :None,
          ord('٬')    :None,
          ord('؛')    :None,
          ord(':')   :None,
          ord('”')     :None,
          ord('_')     :None,
          ord('…')     :None,
          ord('،')     :None,
          ord('ء')     :None,
          ord('”')     :None,
          ord('_')     :None,
          ord('…')     :None,
          ord('ء')     :None,
          ord('؟')     :None,
          ord('!')     :None,
          ord('،')     :None,
          ord('“')     :None,
          ord('ْ')       :None,
          ord(',')     :None,
          ord('\t')     :None,
          ord('\n')     :None,
          ord('-')     :None,
          ord('ٔ')      :None,
          ord('–')    :None
    }    

train_ds["sentence"] = train_ds.sentence.apply(lambda x : x.translate(arabic_to_persian))
train_ds["sentence"] = train_ds.sentence.apply(lambda x : x.translate(special_chars_clear))
train_ds["sentence"] = train_ds.sentence.apply(lambda x : x.translate(char_type_swap))
train_ds["sentence"] = train_ds.sentence.apply(lambda x : x.replace("=", "مساوی"))


validation_ds["sentence"] = validation_ds.sentence.apply(lambda x : x.translate(arabic_to_persian))
validation_ds["sentence"] = validation_ds.sentence.apply(lambda x : x.translate(special_chars_clear))
validation_ds["sentence"] = validation_ds.sentence.apply(lambda x : x.translate(char_type_swap))
validation_ds["sentence"] = validation_ds.sentence.apply(lambda x : x.replace("=", "مساوی"))

since out data is big we use half of our datas for training

In [ ]:
import pandas as pd

train_ds = pd.read_csv("/content/drive/MyDrive/train.tsv", delimiter="\t")
validation_ds = pd.read_csv("/content/drive/MyDrive/dev.tsv", delimiter="\t")

train_ds = train_ds.sample(int(train_ds.shape[0]/2))
validation_ds = validation_ds.sample(int(validation_ds.shape[0]/2))

creating data manifests as required by nemo models

In [ ]:
import os
import json
import librosa

train_manifest = '/content/drive/MyDrive/train_manifest.json'
test_manifest = '/content/drive/MyDrive/test_manifest.json'
dev_manifest =  '/content/drive/MyDrive/dev_manifest.json'

# Function to build a manifest
def build_manifest(meta_data, dir_path, manifest_path):
        with open(manifest_path, 'w') as fout:
            for file_id in meta_data.index.to_numpy():
                file_name = meta_data[meta_data.index == file_id].path.to_numpy()[0].split(".")[0]
                transcript = meta_data[meta_data.index == file_id].sentence.to_numpy()[0]
                audio_path = os.path.join(dir_path, file_name + ".wav")
                duration = librosa.core.get_duration(filename=audio_path)
               
                # Write the metadata to the manifest
                metadata = {
                    "audio_filepath": audio_path,
                    "duration": duration,
                    "text": transcript
                }
                json.dump(metadata, fout)
                fout.write('\n')
                
build_manifest(train_ds, "/content/drive/MyDrive/train_wav", train_manifest)
build_manifest(test_ds, "/content/drive/MyDrive/test_wav", test_manifest)
build_manifest(validation_ds, "/content/drive/MyDrive/dev_wav", dev_manifest)